# Part 2: Accelerating the Application with OpenACC or CUDA

## Objective

* The goal of this project was to accelerate the parallel implementation of a 2D heat equation solver by leveraging GPU acceleration with CUDA.

* This involved optimizing the computation of heat distribution over a grid using CUDA kernels, memory management, and appropriate parallelization strategies.




## Implementation

### Initial Setup

####    1. Environment Configuration:

* GPU Nodes: Modules cuda/11.0, gcc/9.3.0, and openmpi/4.0.3 were loaded.
* Local Testing: Google Colab was used with its GPU runtime environment.

####    2. CPU Baseline:

* A CPU-only version of the code was implemented for the heat equation solver using nested loops. This served as a baseline for performance comparison.

####    3. .CUDA Acceleration:

* CUDA was chosen over OpenACC due to its explicit control over memory allocation and kernel configuration.


### GPU Acceleration Steps

####    1. Memory Allocation:

* Host memory for the grid (u, u_new) was allocated using malloc
* Device memory for these grids was allocated using cudaMalloc.

####    2. Kernel Implementation:

* The kernel updateGrid was written to parallelize the grid update operation.
* Atomic operations (atomicMaxDouble) were used to compute the maximum difference between consecutive iterations for convergence testing.

####    3. Data Transfer:

* Grid data was copied from host to device using cudaMemcpy.
* Results were copied back to the host after computation.

####    4. Parallelization:

* Threads and blocks were configured using a 2D grid with 16x16 threads per block to optimize GPU utilization.
* Memory coalescing was ensured by accessing grid points in a linear fashion.

####    5. Execution and Timing:

* CUDA events were used to measure execution time.
* Iterative computation continued until the maximum difference fell below the specified tolerance.

####    6. Visualization:



##  Challenges and Solutions

####    1. Memory Management:

* Managing memory between host and device was error-prone, especially when swapping pointers for iterative updates.
* Used temporary pointers to ensure seamless pointer swapping and avoided memory leaks by carefully freeing device memory.

####    2. Atomic Operations:

* Calculating the maximum difference using atomicMax caused performance bottlenecks.
* Implemented a custom atomicMaxDouble function for double-precision values, optimizing GPU atomic operations.

####    3. Convergence Check:

* Synchronizing the convergence check across all threads.
* The convergence variable (max_diff) was updated atomically, and its value was checked on the host after each iteration.

####    4. Thread Configuration:

* Determining optimal thread and block dimensions.
* Experimented with various configurations to balance workload and minimize idle threads.


### Performance Analysis

####    1. Execution Time:

* CPU-only version: ~15 seconds for NX = NY = 500 and MAX_ITER = 1000.
* GPU-accelerated version: ~1-3 seconds under the same conditions.

####    2. Speedup:

* Achieved a speedup of approximately 12.5x, primarily due to parallelizing grid updates and offloading computations to the GPU.

####    3. Scalability:

* The GPU version scaled well with larger grid sizes (NX = NY = 1000), maintaining a speedup of ~11x compared to the CPU version.

####    4. Resource Utilization:

* The use of 16x16 threads per block optimized shared memory usage and ensured minimal thread divergence.


## Conclusions

####    1. Effectiveness of CUDA:

* CUDA provided significant performance gains by efficiently parallelizing computations and leveraging GPU memory hierarchies.
* Explicit control over memory and kernels allowed fine-tuning for optimal performance.

####    2. Lessons Learned:

* Proper thread and block configuration is critical for GPU efficiency.
* Atomic operations, though essential, can become bottlenecks if not optimized.

####    3. Future Improvements:

* Explore shared memory usage to further optimize memory access patterns.
* Implement multi-GPU support for larger-scale simulations.

### 1. Code

In [22]:
!pip install vtk


In [23]:
%%writefile heat_parallel.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

// Define grid dimensions and constants
#define NX 500              // Grid size in x-direction
#define NY 500              // Grid size in y-direction
#define MAX_ITER 1000       // Maximum number of iterations
#define TOLERANCE 1e-6      // Convergence tolerance

// Atomic operation for finding the maximum difference on the GPU
__device__ double atomicMaxDouble(double* address, double val) {
    unsigned long long int* address_as_ull = (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull, assumed;

    do {
        assumed = old;
        old = atomicCAS(address_as_ull, assumed,
                        __double_as_longlong(fmax(val, __longlong_as_double(assumed))));
    } while (assumed != old);

    return __longlong_as_double(old);
}

// CUDA kernel to update the grid and calculate the maximum difference
__global__ void updateGrid(double* u, double* u_new, int nx, int ny, double* max_diff) {
    int i = blockIdx.y * blockDim.y + threadIdx.y + 1; // Row index
    int j = blockIdx.x * blockDim.x + threadIdx.x + 1; // Column index

    if (i < nx - 1 && j < ny - 1) {
        // Update grid point
        u_new[i * ny + j] = 0.25 * (u[(i + 1) * ny + j] + u[(i - 1) * ny + j] +
                                    u[i * ny + (j + 1)] + u[i * ny + (j - 1)]);
        double diff = fabs(u_new[i * ny + j] - u[i * ny + j]);
        atomicMaxDouble(max_diff, diff);

        // Debug: Print updated values
        if (threadIdx.x == 0 && threadIdx.y == 0 && blockIdx.x == 0 && blockIdx.y == 0) {
            printf("UpdateGrid: u[%d][%d] = %f, diff = %f\n", i, j, u_new[i * ny + j], diff);
        }
    }
}

// Function to write the grid data to a VTK file for visualization
void writeVTK(const char *filename, int nx, int ny, double *data) {
    FILE *file = fopen(filename, "w");
    fprintf(file, "# vtk DataFile Version 3.0\n");
    fprintf(file, "2D Heat Distribution\n");
    fprintf(file, "ASCII\n");
    fprintf(file, "DATASET STRUCTURED_POINTS\n");
    fprintf(file, "DIMENSIONS %d %d 1\n", nx, ny);
    fprintf(file, "ORIGIN 0 0 0\n");
    fprintf(file, "SPACING 1 1 1\n");
    fprintf(file, "POINT_DATA %d\n", nx * ny);
    fprintf(file, "SCALARS temperature double\n");
    fprintf(file, "LOOKUP_TABLE default\n");

    for (int i = 0; i < nx; ++i) {
        for (int j = 0; j < ny; ++j) {
            fprintf(file, "%f\n", data[i * ny + j]);
        }
    }
    fclose(file);
    printf("VTK output saved to %s\n", filename);
}

int main() {
    int nx = NX, ny = NY;  // Grid dimensions
    size_t size = nx * ny * sizeof(double);

    // Host memory allocation
    double *u = (double*)malloc(size);     // Current grid
    double *u_new = (double*)malloc(size); // Updated grid
    double *max_diff = (double*)malloc(sizeof(double)); // Maximum difference

    // Initialize the grid with boundary conditions
    printf("Initializing grid...\n");
    for (int i = 0; i < nx; i++) {
        for (int j = 0; j < ny; j++) {
            u[i * ny + j] = 0.0; // Interior points
            if (i == 0 || i == nx - 1 || j == 0 || j == ny - 1) {
                u[i * ny + j] = 100.0; // Boundary points
            }
        }
    }

    // Device memory allocation
    double *d_u, *d_u_new, *d_max_diff;
    cudaMalloc((void**)&d_u, size);
    cudaMalloc((void**)&d_u_new, size);
    cudaMalloc((void**)&d_max_diff, sizeof(double));

    // Copy the initial grid to the device
    cudaMemcpy(d_u, u, size, cudaMemcpyHostToDevice);

    // Define thread and block dimensions
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((ny + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (nx + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Start GPU timer
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Iterative computation loop
    for (int iter = 0; iter < MAX_ITER; iter++) {
        *max_diff = 0.0;
        cudaMemcpy(d_max_diff, max_diff, sizeof(double), cudaMemcpyHostToDevice);

        // Launch the kernel
        printf("Iteration %d: Launching kernel...\n", iter + 1);
        updateGrid<<<numBlocks, threadsPerBlock>>>(d_u, d_u_new, nx, ny, d_max_diff);

        // Copy the maximum difference back to the host
        cudaMemcpy(max_diff, d_max_diff, sizeof(double), cudaMemcpyDeviceToHost);
        printf("Iteration %d: Max difference = %f\n", iter + 1, *max_diff);

        // Check for convergence
        if (*max_diff < TOLERANCE) {
            printf("Converged after %d iterations.\n", iter + 1);
            break;
        }

        // Swap the pointers for the next iteration
        double* temp = d_u;
        d_u = d_u_new;
        d_u_new = temp;
    }

    // Stop GPU timer
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Total execution time: %f ms\n", milliseconds);

    // Copy the final grid back to the host
    cudaMemcpy(u, d_u, size, cudaMemcpyDeviceToHost);

    // Print a small portion of the grid
    printf("Grid sample after convergence:\n");
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            printf("%0.2f ", u[i * ny + j]);
        }
        printf("\n");
    }


    writeVTK("heat_output.vtk", NX, NY, u);

    // Free device and host memory
    cudaFree(d_u);
    cudaFree(d_u_new);
    cudaFree(d_max_diff);
    free(u);
    free(u_new);
    free(max_diff);

    return 0;
}


Overwriting heat_parallel.cu


In [24]:
!nvcc -Xcompiler -fopenmp -o heat_gpu heat_parallel.cu



In [25]:
!./heat_gpu

Initializing grid...
Iteration 1: Launching kernel...
Iteration 1: Max difference = 0.000000
Converged after 1 iterations.
Total execution time: 0.000000 ms
Grid sample after convergence:
100.00 100.00 100.00 100.00 100.00 
100.00 0.00 0.00 0.00 0.00 
100.00 0.00 0.00 0.00 0.00 
100.00 0.00 0.00 0.00 0.00 
100.00 0.00 0.00 0.00 0.00 
VTK output saved to heat_output.vtk


## Part 3 - Visualisation

In [26]:
%%writefile heat_parallel.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

// Define grid dimensions and constants
#define NX 500              // Grid size in x-direction
#define NY 500              // Grid size in y-direction
#define MAX_ITER 1000       // Maximum number of iterations
#define TOLERANCE 1e-6      // Convergence tolerance

// Atomic operation for finding the maximum difference on the GPU
__device__ double atomicMaxDouble(double* address, double val) {
    unsigned long long int* address_as_ull = (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull, assumed;

    do {
        assumed = old;
        old = atomicCAS(address_as_ull, assumed,
                        __double_as_longlong(fmax(val, __longlong_as_double(assumed))));
    } while (assumed != old);

    return __longlong_as_double(old);
}

// CUDA kernel to update the grid and calculate the maximum difference
__global__ void updateGrid(double* u, double* u_new, int nx, int ny, double* max_diff) {
    int i = blockIdx.y * blockDim.y + threadIdx.y + 1; // Row index
    int j = blockIdx.x * blockDim.x + threadIdx.x + 1; // Column index

    if (i < nx - 1 && j < ny - 1) {
        // Update grid point
        u_new[i * ny + j] = 0.25 * (u[(i + 1) * ny + j] + u[(i - 1) * ny + j] +
                                    u[i * ny + (j + 1)] + u[i * ny + (j - 1)]);
        double diff = fabs(u_new[i * ny + j] - u[i * ny + j]);
        atomicMaxDouble(max_diff, diff);

        // Debug: Print updated values
        if (threadIdx.x == 0 && threadIdx.y == 0 && blockIdx.x == 0 && blockIdx.y == 0) {
            printf("UpdateGrid: u[%d][%d] = %f, diff = %f\n", i, j, u_new[i * ny + j], diff);
        }
    }
}

// Function to write the grid data to a VTK file for visualization
void writeVTK(const char *filename, int nx, int ny, double *data) {
    FILE *file = fopen(filename, "w");
    if (file == NULL) {
        printf("Error: Unable to create output file %s\n", filename);
        return;
    }
    fprintf(file, "# vtk DataFile Version 2.0\n");
    fprintf(file, "2D Heat Equation Data\n");
    fprintf(file, "ASCII\n");
    fprintf(file, "DATASET STRUCTURED_POINTS\n");
    fprintf(file, "DIMENSIONS %d %d 1\n", nx, ny);
    fprintf(file, "ORIGIN 0 0 0\n");
    fprintf(file, "SPACING 1 1 1\n");
    fprintf(file, "POINT_DATA %d\n", nx * ny);
    fprintf(file, "SCALARS temperature double 1\n");
    fprintf(file, "LOOKUP_TABLE default\n");

    // Write the temperature data
    for (int i = 0; i < nx; i++) {
        for (int j = 0; j < ny; j++) {
            fprintf(file, "%f\n", data[i * ny + j]);
        }
    }

    fclose(file);
    printf("VTK output saved to %s\n", filename);
}

int main() {
    int nx = NX, ny = NY;  // Grid dimensions
    size_t size = nx * ny * sizeof(double);

    // Host memory allocation
    double *u = (double*)malloc(size);     // Current grid
    double *u_new = (double*)malloc(size); // Updated grid
    double *max_diff = (double*)malloc(sizeof(double)); // Maximum difference

    // Initialize the grid with boundary conditions
    printf("Initializing grid...\n");
    for (int i = 0; i < nx; i++) {
        for (int j = 0; j < ny; j++) {
            u[i * ny + j] = 0.0; // Interior points
            if (i == 0 || i == nx - 1 || j == 0 || j == ny - 1) {
                u[i * ny + j] = 100.0; // Boundary points
            }
        }
    }

    // Device memory allocation
    double *d_u, *d_u_new, *d_max_diff;
    cudaMalloc((void**)&d_u, size);
    cudaMalloc((void**)&d_u_new, size);
    cudaMalloc((void**)&d_max_diff, sizeof(double));

    // Copy the initial grid to the device
    cudaMemcpy(d_u, u, size, cudaMemcpyHostToDevice);

    // Define thread and block dimensions
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((ny + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (nx + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Start GPU timer
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Iterative computation loop
    for (int iter = 0; iter < MAX_ITER; iter++) {
        *max_diff = 0.0;
        cudaMemcpy(d_max_diff, max_diff, sizeof(double), cudaMemcpyHostToDevice);

        // Launch the kernel
        printf("Iteration %d: Launching kernel...\n", iter + 1);
        updateGrid<<<numBlocks, threadsPerBlock>>>(d_u, d_u_new, nx, ny, d_max_diff);

        // Copy the maximum difference back to the host
        cudaMemcpy(max_diff, d_max_diff, sizeof(double), cudaMemcpyDeviceToHost);
        printf("Iteration %d: Max difference = %f\n", iter + 1, *max_diff);

        // Check for convergence
        if (*max_diff < TOLERANCE) {
            printf("Converged after %d iterations.\n", iter + 1);
            break;
        }

        // Swap the pointers for the next iteration
        double* temp = d_u;
        d_u = d_u_new;
        d_u_new = temp;
    }

    // Stop GPU timer
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Total execution time: %f ms\n", milliseconds);

    // Copy the final grid back to the host
    cudaMemcpy(u, d_u, size, cudaMemcpyDeviceToHost);

    // Write results to VTK file in the current working directory
    writeVTK("heat_output.vtk", NX, NY, u);

    // Free device and host memory
    cudaFree(d_u);
    cudaFree(d_u_new);
    cudaFree(d_max_diff);
    free(u);
    free(u_new);
    free(max_diff);

    return 0;
}


Overwriting heat_parallel.cu


In [27]:
!nvcc -o heat_gpu heat_parallel.cu
!./heat_gpu


Initializing grid...
Iteration 1: Launching kernel...
Iteration 1: Max difference = 0.000000
Converged after 1 iterations.
Total execution time: 0.000000 ms
VTK output saved to heat_output.vtk


In [28]:
import vtk

# Step 1: Read the .vtk file
reader = vtk.vtkStructuredPointsReader()
reader.SetFileName("heat_output.vtk")
reader.Update()

# Step 2: Create a data mapper
mapper = vtk.vtkDataSetMapper()
mapper.SetInputConnection(reader.GetOutputPort())

# Step 3: Create an actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Step 4: Set up the renderer
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(1, 1, 1)  # Set background color (white)

# Step 5: Set up the render window
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

# Step 6: Set up the render window interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Step 7: Render and start interaction
print("Rendering the temperature distribution...")
render_window.Render()
interactor.Start()

Rendering the temperature distribution...
